# Interfaz Grafica



In [2]:
#Importar librerias

import tkinter as tk
from tkinter import filedialog
import pandas as pd

from joblib import load

from sklearn.preprocessing import Normalizer

In [3]:
#Cargar los modelos

#Modelo ClasificacionLDA
modeloClasificacionCFG = load('./modelosPKL/ModeloClasificacionCrudosFondosGasesLDA.pkl')
modeloPca = load('./modelosPKL/modelo_pca.joblib')

#Modelo RegresionCCR SVR
modeloCCR= load('./modelosPKL/Regrecion%CCR_SVR.pkl')

#Regresion_Saturados_SVR.pkl
modeloSaturados= load('./modelosPKL/Regresion_Saturados_SVR.pkl')
#Regresion_Aromaticos_SVR.pkl
modeloAromaticos= load('./modelosPKL/Regresion_Aromaticos_PLS.pkl')
#Regresion_Resinas_SVR.pkl
modeloResinas= load('./modelosPKL/Regresion_Resinas_SVR.pkl')
#Regresion_Asfaltenos_SVR.pkl
modeloAsfaltenos= load('./modelosPKL/Regresion_Asfaltenos_SVR.pkl')

#Regresion_ASCI_SVR.pkl
modeloASCI= load('./modelosPKL/Classifier_ASCI_SVR.pkl')


## Creacion GUI

In [4]:
## @version 16 mayo 2023

import tkinter as tk
import numpy as np


#Global var
datosClasificacion = None
datosCCR = None
datosSARA = None
datosASCI = None

## Regresar al modulo Root
def volver_a_principal():
    # Mostrar la ventana principal
    root.deiconify()
    # Ocultar la ventana de clasificación
    VentanaCCR.withdraw()
    

## Ventana Clasificacion
def VentanaClasificacion():
    # Crear una nueva ventana
    VentanaClasificacion = tk.Toplevel(root)
    VentanaClasificacion.title("Clasificacion de Fondos/Crudos/Gases")
    VentanaClasificacion.geometry("800x600")
    
    #Agregar Titulo
    title_label = tk.Label(VentanaClasificacion, text="GUI", font=("Arial", 30, "bold"), fg="orange")
    title_label.pack(side="top", pady=20)
    
    # Agregar el subtitulo
    subtitle_label = tk.Label(VentanaClasificacion, text="Clasificación Crudos/Fondos/Gases", font=("Arial", 16, "bold"))
    subtitle_label.pack(side="top", pady=10)
    
    #Funcion Cargar CSV
    def cargar_csv():
        # Mostrar cuadro de diálogo para seleccionar el archivo csv
        archivo = filedialog.askopenfilename(filetypes=[("Archivos CSV", "*.csv")])
        # Cargar el archivo con pandas
        global datosClasificacion
        datosClasificacion = pd.read_csv(archivo)
        # Obtener los primeros registros del dataset como texto
        texto = datosClasificacion.head(10).to_string(index=False)
        # Mostrar los registros en el widget Text
        resultados_text.delete("1.0", tk.END)
        resultados_text.insert(tk.END, texto)
    
    #Funcion Prediccion del modelo Clasificacion
    def predecir_csv():
        global datosClasificacion
        datosClasificacion=datosClasificacion.transpose()
        datosClasificacionPCA = modeloPca.transform(datosClasificacion)
        prediccionesClasificacion = modeloClasificacionCFG.predict(datosClasificacionPCA)

        
        if (prediccionesClasificacion == 2):
            texto =("Sus datos son un crudo")
        elif(prediccionesClasificacion ==1):
            texto =("Sus datos son un fondo")
        else :
            texto =("Sus datos son un gas")
        # Mostrar los registros en el widget Text
        prediction_text.delete("1.0", tk.END)
        prediction_text.insert(tk.END, texto)
        
        
    # Agregar los botones
    cargar_btn = tk.Button(VentanaClasificacion, text="Cargar Archivos", font=("Arial", 12), command=cargar_csv)
    cargar_btn.pack(pady=10)
    
    predecir_btn = tk.Button(VentanaClasificacion, text="Predecir", font=("Arial", 12), command= predecir_csv)
    predecir_btn.pack(pady=10)
    
    # Crear un widget tk.Text() para mostrar los primeros n registros del archivo CSV
    resultados_text = tk.Text(VentanaClasificacion, height=5, font=("Arial", 12))
    resultados_text.pack(padx=10, pady=10)
    # Crear un widget tk.Text() para mostrar la prediccion del modelo
    prediction_text = tk.Text(VentanaClasificacion,height=3, font=("Arial", 12))
    prediction_text.pack()

    
    volver_btn = tk.Button(VentanaClasificacion, text="volver", font=("Arial", 12), 
                           command=lambda: [VentanaClasificacion.withdraw(), root.deiconify()])
    volver_btn.pack(side="left",pady=100)
    volver_btn.place(x=50, y=500)


    # Ocultar la ventana actual
    root.withdraw()

    # Mostrar la nueva ventana
    VentanaClasificacion.mainloop()
    
    
## Ventana CCR
def VentanaCCR():
    # Crear una nueva ventana
    VentanaCCR = tk.Toplevel(root)
    VentanaCCR.title("Predicción CCR")
    VentanaCCR.geometry("800x600")
    
    # Agregar Título
    title_label = tk.Label(VentanaCCR, text="GUI", font=("Arial", 30, "bold"), fg="orange")
    title_label.pack(side="top", pady=20)
    
    # Agregar el subtítulo
    subtitle_label = tk.Label(VentanaCCR, text="Predicción CCR", font=("Arial", 16, "bold"))
    subtitle_label.pack(side="top", pady=10)

    # Función Cargar CSV
    def cargar_csv():
        # Mostrar cuadro de diálogo para seleccionar el archivo csv
        archivo = filedialog.askopenfilename(filetypes=[("Archivos CSV", "*.csv")])
        # Cargar el archivo con pandas
        global datosCCR
        datosCCR = pd.read_csv(archivo, sep="\t", decimal=",")
        # Obtener los primeros registros del dataset como texto
        texto = datosCCR.head(10).to_string(index=False)
        # Mostrar los registros en el widget Text
        resultados_text.delete("1.0", tk.END)
        resultados_text.insert(tk.END, texto)
        
    # Función Predicción del modelo CCR
    def predecirCCR_csv():
        global datosCCR
        datosCCR = datosCCR.transpose()
        datosCCR = Normalizer().fit_transform(datosCCR)
        prediccionesCCR = modeloCCR.predict(datosCCR)

        texto = ("Su crudo cuenta con un valor CCR de ", prediccionesCCR)
        # Mostrar los registros en el widget Text
        prediction_text.delete("1.0", tk.END)
        prediction_text.insert(tk.END, texto)
    
    # Agregar los botones
    cargar_btn = tk.Button(VentanaCCR, text="Cargar Archivos", font=("Arial", 12), command=cargar_csv)
    cargar_btn.pack(pady=10)

    predecir_btn = tk.Button(VentanaCCR, text="Predecir CCR", font=("Arial", 12), command=predecirCCR_csv)
    predecir_btn.pack(pady=10)
    
    # Crear un widget tk.Text() para mostrar los primeros n registros del archivo CSV
    resultados_text = tk.Text(VentanaCCR, height=5, font=("Arial", 12))
    resultados_text.pack(padx=10, pady=10)
    # Crear un widget tk.Text() para mostrar la predicción del modelo
    prediction_text = tk.Text(VentanaCCR, height=3, font=("Arial", 12))
    prediction_text.pack()
    
    volver_btn = tk.Button(VentanaCCR, text="Volver", font=("Arial", 12), 
                           command=lambda: [VentanaCCR.withdraw(), root.deiconify()])
    volver_btn.pack(side="left", padx=50, pady=70)
    volver_btn.place(x=50, y=500)
    
    # Ocultar la ventana actual
    root.withdraw()

    # Mostrar la nueva ventana
    VentanaCCR.mainloop()

## Ventana SARA
def VentanaSARA():
    # Crear una nueva ventana
    VentanaSARA = tk.Toplevel(root)
    VentanaSARA.title("Prediccion Sara")
    VentanaSARA.geometry("800x600")
    
    #Agregar Titulo
    title_label = tk.Label(VentanaSARA, text="GUI", font=("Arial", 30, "bold"), fg="orange")
    title_label.pack(side="top", pady=20)
    
    # Agregar el subtitulo
    subtitle_label = tk.Label(VentanaSARA, text="Prediccion SARA", font=("Arial", 16, "bold"))
    subtitle_label.pack(side="top", pady=10)
    
    #Funcion Cargar CSV
    def cargar_csv():
        # Mostrar cuadro de diálogo para seleccionar el archivo csv
        archivo = filedialog.askopenfilename(filetypes=[("Archivos CSV", "*.csv")])
        # Cargar el archivo con pandas
        global datosSARA
        datosSARA = pd.read_csv(archivo, sep="\t", decimal=",")
        # Obtener los primeros registros del dataset como texto
        texto = datosSARA.head(10).to_string(index=False)
        # Mostrar los registros en el widget Text
        resultados_text.delete("1.0", tk.END)
        resultados_text.insert(tk.END, texto)
    
    #Funcion Prediccion del modelo SARA
    def predecir_csv():
        global datosSARA
        datosSARA=datosSARA.transpose()
        datosSARA = Normalizer().fit_transform(datosSARA)
        prediccionesSaturados = modeloSaturados.predict(datosSARA)
        prediccionesAromaticos = modeloAromaticos.predict(datosSARA)
        prediccionesResinas = modeloResinas.predict(datosSARA)
        prediccionesAsfaltenos = modeloAsfaltenos.predict(datosSARA)


        
        texto = (
                "Su crudo cuenta con un valor de Saturados = " + str(prediccionesSaturados) + "\n" +
                "Su crudo cuenta con un valor de Aromaticos = " + str(prediccionesAromaticos) + "\n" +
                "Su crudo cuenta con un valor de Resinas = " + str(prediccionesResinas) + "\n" +
                "Su crudo cuenta con un valor de Asfaltenos = " + str(prediccionesAsfaltenos) + "\n"
                )

        # Mostrar los registros en el widget Text
        prediction_text.delete("1.0", tk.END)
        prediction_text.insert(tk.END, texto)
        
        
    
    # Agregar los botones
    cargar_btn = tk.Button(VentanaSARA, text="Cargar Archivos", font=("Arial", 12), command=cargar_csv)
    cargar_btn.pack(pady=10)

    predecir_btn = tk.Button(VentanaSARA, text="Predecir SARA", font=("Arial", 12), command= predecir_csv)
    predecir_btn.pack(pady=10)
    
    # Crear un widget tk.Text() para mostrar los primeros n registros del archivo CSV
    resultados_text = tk.Text(VentanaSARA, height=5, font=("Arial", 12))
    resultados_text.pack(padx=10, pady=10)
    # Crear un widget tk.Text() para mostrar la prediccion del modelo
    prediction_text = tk.Text(VentanaSARA,height=4, font=("Arial", 12))
    prediction_text.pack()
    
    volver_btn = tk.Button(VentanaSARA, text="volver", font=("Arial", 12), 
                           command=lambda: [VentanaSARA.withdraw(), root.deiconify()])
    volver_btn.pack(side="left", padx=50,pady=100)
    volver_btn.place(x=50, y=500)

    # Ocultar la ventana actual
    root.withdraw()

    # Mostrar la nueva ventana
    VentanaSARA.mainloop()
    
    
## Ventana ASCI
def VentanaASCI():
    # Crear una nueva ventana
    VentanaASCI = tk.Toplevel(root)
    VentanaASCI.title("Prediccion ASCI")
    VentanaASCI.geometry("800x600")
    
    #Agregar Titulo
    title_label = tk.Label(VentanaASCI, text="GUI", font=("Arial", 30, "bold"), fg="orange")
    title_label.pack(side="top", pady=20)
    
    # Agregar el subtitulo
    subtitle_label = tk.Label(VentanaASCI, text="Prediccion ASCI", font=("Arial", 16, "bold"))
    subtitle_label.pack(side="top", pady=10)

    #Funcion Cargar CSV
    def cargar_csv():
        # Mostrar cuadro de diálogo para seleccionar el archivo csv
        archivo = filedialog.askopenfilename(filetypes=[("Archivos CSV", "*.csv")])
        # Cargar el archivo con pandas
        global datosASCI
        datosASCI = pd.read_csv(archivo, sep="\t", decimal=",")
        # Obtener los primeros registros del dataset como texto
        texto = datosASCI.head(10).to_string(index=False)
        # Mostrar los registros en el widget Text
        resultados_text.delete("1.0", tk.END)
        resultados_text.insert(tk.END, texto)
        
    #Funcion Prediccion del modelo ASCI
    def predecirASCI_csv():
        global datosASCI
        datosASCI=datosASCI.transpose()
        datosASCI = Normalizer().fit_transform(datosASCI)
        prediccionesASCI = modeloASCI.predict(datosASCI)

        
        texto =("Su crudo cuenta con un valor ASCI de ", prediccionesASCI )
        # Mostrar los registros en el widget Text
        prediction_text.delete("1.0", tk.END)
        prediction_text.insert(tk.END, texto)
    
    # Agregar los botones
    cargar_btn = tk.Button(VentanaASCI, text="Cargar Archivos", font=("Arial", 12), command=cargar_csv)
    cargar_btn.pack(pady=10)

    predecir_btn = tk.Button(VentanaASCI, text="Predecir ASCI", font=("Arial", 12), command=predecirASCI_csv)
    predecir_btn.pack(pady=10)
    
    # Crear un widget tk.Text() para mostrar los primeros n registros del archivo CSV
    resultados_text = tk.Text(VentanaASCI, height=5, font=("Arial", 12))
    resultados_text.pack(padx=10, pady=10)
    # Crear un widget tk.Text() para mostrar la prediccion del modelo
    prediction_text = tk.Text(VentanaASCI,height=3, font=("Arial", 12))
    prediction_text.pack()
    
    volver_btn = tk.Button(VentanaASCI, text="volver", font=("Arial", 12), 
                           command=lambda: [VentanaASCI.withdraw(), root.deiconify()])
    volver_btn.pack(side="left", padx=50,pady=70)
    volver_btn.place(x=50, y=500)
    
    # Ocultar la ventana actual
    root.withdraw()

    # Mostrar la nueva ventana
    VentanaASCI.mainloop()

# Cerrar Ventana
# def cerrar_programa():
    # root.quit()
    # root.protocol("WM_DELETE_WINDOW", root.quit)


    
# Crear ventana principal
root = tk.Tk()
root.geometry("800x600")
root.title("GUI")

# Agregar título en naranja y negrita centrado
title_label = tk.Label(root, text="GUI", font=("Arial", 30, "bold"), fg="orange")
title_label.pack(side="top", pady=20)

# Agregar subtítulo en negrita
subtitle_label = tk.Label(root, text="Seleccione la característica que quiere predecir", font=("Arial", 16, "bold"))
subtitle_label.pack(side="top", pady=10)

# Agregar botones equidistantes
button1 = tk.Button(root, text="Clasificación", font=("Arial", 12),command=VentanaClasificacion)
button1.pack()

button2 = tk.Button(root, text="Prediccion de % CCR", font=("Arial", 12),command=VentanaCCR)
button2.pack(side="top", padx=0, pady=20)

button3 = tk.Button(root, text="Predicciones SARA", font=("Arial", 12), command=VentanaSARA)
button3.pack(side="top", padx=0, pady=20) 

button4 = tk.Button(root, text="Predicciones ASCI", font=("Arial", 12), command=VentanaASCI)
button4.pack(side="top", padx=0, pady=20)

# cerrar_btn = tk.Button(root, text="Cerrar", command=cerrar_programa)
# cerrar_btn.pack(side="right", padx=50,pady=80)

# Ejecutar la ventana principal
root.mainloop()


